In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd 
import math
import random
from keras.utils import pad_sequences
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_json('/content/drive/MyDrive/train.json', typ = 'series')

index = df.keys()
log_event=[] #store log information
train = [] #store 1 or 0 for anomaly
all_log_event = {} #all_log_event store all usefull information of log with its classification

#### Extracting usefull information of log ########
j = 0
for i in index:
    temp='{}'.format(j)
    for k in i.split()[8:]:
        if(any(ch.isdigit() for ch in k) == False):
            temp+=k
            temp+=" "
    all_log_event[temp]=df[i]
    j+=1

In [4]:
##### getting random data from all_log_event ########
N = 1000000
ana = [i for i in all_log_event if all_log_event[i] == 'abnormal']
nor = [i for i in all_log_event if all_log_event[i] == 'normal']
log_event = ana
log_event.extend(random.sample( list(all_log_event),N-69692))#N data to train W matrix

In [5]:
print(len(log_event))

1000000


In [6]:
tr = bool(0)#anomaly or normal for every log data
for i in log_event:
    if(all_log_event[i] == 'normal'):
        tr = 0;
    else:
        tr = 1;
    train.append(tr)
    tr = bool(0)

In [7]:
#########Preprocessing on log_event############
#storing log_event into my_data
my_data = []
for i in log_event:
    my_data.append(i)
vectorize = Tokenizer(filters='1234567890')
vectorize.fit_on_texts(my_data)#creating dictionary of unique words
my_data = vectorize.texts_to_sequences(my_data)#replacing words with numbers
total_vocab = sum(len(s) for s in my_data)
word_count = len(vectorize.word_index) + 1
print(word_count)

729


In [9]:
# print(my_data)

In [ ]:
window_size = 10
dim_word = int(word_count/2)
Tf = []
W = []
model = Sequential()
model.add(Dense(1, activation='softmax'))
model.add(Dense(dim_word, activation='relu'))
model.add(Dense(2*window_size, activation='relu'))
model.compile(loss='MeanSquaredError', optimizer='adam')
coded = vectorize.word_index
for i in range(1,1+len(coded)):
    tftemp=0
    inputt = []
    outputt = []
    for k in my_data:
        if(i in k):
            tftemp+=1
            text_len = len(k)
            context_word = []
            idx = k.index(i)#index of target word
            #getting things ready for context
            begin = idx - window_size
            end = idx + window_size + 1
            context_word.append([k[m] for m in range(begin, end) if (0 <= m < text_len and m != idx)])
            contextual = pad_sequences(context_word, maxlen = 2*window_size)
            #done
            #making input as a vector
            inp = pad_sequences([[i,],])
            #only add new values to training dataset
            flag=0
            for j in outputt:
                if (contextual == j).all():
                    flag=1
            if(flag==0):
                outputt.append(contextual)
                inputt.append(inp)
    Tf.append(tftemp)
    #training for given words using training set
    for j in range(len(inputt)):
        # print("hello")
        model.fit(inputt[j],outputt[j],epochs=2,verbose = 0)

In [ ]:
#making other neural network to calculate features of a word (value of hidden layer)
model2 = Sequential()
model2.add(Dense(1, activation='softmax',weights=model.layers[0].get_weights()))
model2.add(Dense(dim_word, activation='softmax'))
###Calculating W Matrix
W.append(model2.predict([[inp2,],]))

In [ ]:
model.summary()

In [ ]:
# print(np.array(W).shape)

In [ ]:
########--Calculation of Tf-Idf--###########
Tf = [i/word_count for i in Tf]
Idf = []
for i in range(1,1+len(coded)):
    idftemp = 0
    for j in my_data:
        if(i in j):
            idftemp+=1
    Idf.append(idftemp)
Idf = [np.log((N+1)/idf) for idf in Idf]
TfIdf = list(np.array(Tf)*np.array(Idf))

In [ ]:
#########--Calculate T.matrix--##########
T =[]
for i in my_data[:100000]:
    temp = []
    for j in range(1,1+len(coded)):
        if(j in i):
            temp.append(TfIdf[j-1])
        else:
            temp.append(0)
    T.append(temp)

In [ ]:
# print(T.shape)

In [ ]:
# print(W)
newW=[]
for i in W:
    newW.append(i[0])

In [ ]:
# print(W)

In [ ]:
##########Calculating E matrix############
E = np.matmul(np.array(T),np.array(newW))

In [ ]:
print(len(train))

In [ ]:
###########Final Neural Network#########
final_model = Sequential()
final_model.add(Dense(dim_word, activation='relu'))
# final_model.add(Dense(100, activation='relu'))
final_model.add(Dense(1, activation='sigmoid'))
final_model.compile(loss='binary_crossentropy', optimizer='adam')
ind =0 
for inp in (E):
    temp = []
    for i in inp:
        temp.append(float(i))
    final_model.fit([temp],[[int(train[ind])]], verbose = 0)
    ind+=1

In [ ]:
# final_model.fit(Etrain,trained,epoch = 4)

In [ ]:
final_model.summary()

In [ ]:
###########Creating test case#############
Ntest = 100000
test_data = random.sample(list(all_log_event),Ntest)
test = []
tr = bool(0)#anomaly or normal for every log data
for i in test_data:
    if(all_log_event[i] == 'normal'):
        tr = 0;
    else:
        tr = 1;
    test.append(tr)
    tr = bool(0)
# vectorize = Tokenizer(filters='1234567890')
test_data = vectorize.texts_to_sequences(test_data)#replacing words with numbers

Ttest =[]
for i in test_data:
    temp = []
    for j in range(1,1+len(coded)):
        if(j in i):
            temp.append(TfIdf[j-1])
        else:
            temp.append(0)
    Ttest.append(temp)

In [ ]:
###creating a list of indices where there is anommaly
my_list = np.array(test) 
indices = np.where(my_list == 1)[0]
print(indices)

In [ ]:
#####Creating a E test matrix############
Etest = np.matmul(np.array(Ttest),np.array(newW))

In [ ]:
#####Predicting#######
# ans = final_model.predict(Etest)

In [ ]:
######Calculating TP,TN, FP, FN and F1 score###########
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0
thre=0.01070902

for count,i in enumerate(ans):
    if i>=thre:#predicted anomaly or predicted neg
        if(test[count] == 0):#actual normal
            false_negative +=1
        else:#actual abnormal
            true_negative +=1
    else:#predicted normal or precidicted pos
        if(test[count] == 0):#actual normal
            true_positive +=1
        else:#actual abnormal
            false_positive +=1       

In [ ]:
########Evaluation of model###########
print(true_positive," ", true_negative," ", false_positive," ",false_negative)
precision = true_positive/(true_positive + false_positive)
recall = true_positive/(true_positive + false_negative)

print(precision," ",recall)
f1 = 2*precision*recall/(precision+recall)
print(f1)

In [ ]:
######################################################################################################

In [ ]:
df = pd.read_csv('/kaggle/input/convolve-train/test.csv')
test_data = list(df[' Log'])

In [ ]:
# print(test_data)

In [ ]:
#######Preprocessing Test Data############
ptest_data = []
j = 0
for i in test_data:
    temp=""
    for k in i.split()[8:]:
        if(any(ch.isdigit() for ch in k) == False):
            temp+=k
            temp+=" "
    ptest_data.append(temp)
    j+=1
test_data = ptest_data

In [ ]:
###########Matrix T#############

# vectorize = Tokenizer(filters='1234567890')
test_data = vectorize.texts_to_sequences(test_data)#replacing words with numbers

Ttest =[]
for i in test_data:
    temp = []
    for j in range(1,1+len(coded)):
        if(j in i):
            temp.append(TfIdf[j-1])
        else:
            temp.append(0)
    Ttest.append(temp)

In [ ]:
#####Creating a E test matrix############
Etest = np.matmul(np.array(Ttest),np.array(newW))

In [ ]:
#####Predicting#######
ans = final_model.predict(Etest)

In [ ]:
######binary classification######
bans = []
for i in ans:
    if i>=thre:#predicted anomaly or predicted neg
        bans.append("abnormal" )
    else:#predicted normal or precidicted pos
        bans.append("normal")    

In [ ]:
submission = {'ID':list(df['ID']),' Label':bans}
df = pd.DataFrame(submission)

In [ ]:
print(df)

In [ ]:
df.to_csv('sub2.csv', index=False)